In [ ]:
from __future__ import print_function, division, unicode_literals, absolute_import

import random
from nltk.corpus import wordnet as wn
import pandas as pd

In [ ]:
def transitive_closure(synsets):

    hypernyms = set([])
    for s in synsets:
        paths = s.hypernym_paths()
        for path in paths:
            hypernyms.update((s,h) for h in path[1:] if h.pos() == 'n')
    return hypernyms

In [ ]:
def get_embds(target='mammal.n.01', shuffle=True):

    target = wn.synset(target)
    print('target:', target)

    words = wn.words()

    nouns = set([])
    for word in words:
        nouns.update(wn.synsets(word, pos='n'))

    print( len(nouns), 'nouns')

    hypernyms = []
    hypernyms_connections = []
    for noun in nouns:
        paths = noun.hypernym_paths()
        for path in paths:
            try:
                pos = path.index(target)
                for i in range(pos, len(path)-1):
                    hypernyms.append((noun, path[i]))
                    hypernyms_connections.append((path[i], path[i+1]))
                    
            except Exception:
                continue
        
    hypernyms = list(set(hypernyms))
    print( len(hypernyms), 'hypernyms' )
    
    
    if not shuffle:
        random.shuffle(hypernyms)
    
    hypernyms = [(n1.name(), n2.name()) for n1, n2 in hypernyms]
    hypernyms_df = pd.DataFrame(hypernyms, columns=['A', 'B'])
    hypernyms_df.index.name = "id"
    
    hypernyms_con = [(n1.name(), n2.name()) for n1, n2 in hypernyms_connections]
    hypernyms_con_df = pd.DataFrame(hypernyms_con, columns=['from', 'to'])
    hypernyms_con_df.index.name = "id"    
    return hypernyms_df, hypernyms_con_df

hypernyms_df, hypernyms_con_df = get_embds(shuffle=True)

In [ ]:
hypernyms_df.to_csv("data/mammal.csv")
hypernyms_con_df.to_csv("data/mammal_connections.csv")

In [ ]:
len(hypernyms_df.A.unique())

In [ ]:
hypernyms_df[:5]